In [1]:
# Install and import required libraries
import pandas as pd
import yfinance as yf
import os
from datetime import datetime, timedelta
import glob

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
def get_currency_symbol(currency_code):
    """Map currency codes to Yahoo Finance symbols"""
    # Yahoo Finance uses format like EURUSD=X for currency pairs
    if currency_code == 'USD':
        return None  # USD is the base currency

    return f"{currency_code}USD=X"

def get_last_date_from_csv(filepath):
    """Get the last date from a CSV file"""
    try:
        df = pd.read_csv(filepath)
        if len(df) == 0:
            return None

        # Convert date column to datetime
        df['Date'] = pd.to_datetime(df['Date'])
        last_date = df['Date'].max()
        return last_date
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

def fetch_missing_rates(currency_code, last_date, end_date):
    """Fetch missing exchange rates from Yahoo Finance"""
    symbol = get_currency_symbol(currency_code)
    if symbol is None:
        print(f"Skipping USD (base currency)")
        return None

    try:
        # Add one day to last_date to avoid duplicates
        start_date = last_date + timedelta(days=1)

        # Skip if no missing dates
        if start_date > end_date:
            print(f"No missing dates for {currency_code}")
            return None

        print(f"Fetching {currency_code} rates from {start_date.date()} to {end_date.date()}")

        # Fetch data from Yahoo Finance
        ticker = yf.Ticker(symbol)
        hist = ticker.history(start=start_date, end=end_date + timedelta(days=1))

        if hist.empty:
            print(f"No data available for {symbol}")
            return None

        # Use the 'Close' price as the exchange rate
        rates_df = pd.DataFrame({
            'Date': hist.index,
            'Rate': hist['Close']
        })

        # Reset index and ensure Date is in the right format
        rates_df.reset_index(drop=True, inplace=True)
        rates_df['Date'] = pd.to_datetime(rates_df['Date'])

        return rates_df

    except Exception as e:
        print(f"Error fetching data for {currency_code}: {e}")
        return None

# Test the functions
print("Functions defined successfully")

Functions defined successfully


In [3]:
# Set the directory path and target date
exchange_rates_dir = '/Users/connor.sullivan/mma/823/exchange_rates'
today = datetime.now().date()
target_date = datetime(2025, 10, 28).date()  # Today's date

print(f"Processing exchange rate files from {exchange_rates_dir}")
print(f"Target end date: {target_date}")

# Get all CSV files in the exchange_rates directory
csv_files = glob.glob(os.path.join(exchange_rates_dir, "*_USD_rates.csv"))
print(f"Found {len(csv_files)} exchange rate files to process")

# Show the files we'll process
for file in csv_files[:5]:  # Show first 5 files
    filename = os.path.basename(file)
    currency_code = filename.split('_')[0]
    print(f"  {filename} -> Currency: {currency_code}")

if len(csv_files) > 5:
    print(f"  ... and {len(csv_files) - 5} more files")

Processing exchange rate files from /Users/connor.sullivan/mma/823/exchange_rates
Target end date: 2025-10-28
Found 56 exchange rate files to process
  JOD_USD_rates.csv -> Currency: JOD
  JPY_USD_rates.csv -> Currency: JPY
  NOK_USD_rates.csv -> Currency: NOK
  PEN_USD_rates.csv -> Currency: PEN
  SVC_USD_rates.csv -> Currency: SVC
  ... and 51 more files


In [4]:
# Process each exchange rate file
updated_files = []
failed_files = []

for csv_file in csv_files:
    filename = os.path.basename(csv_file)
    currency_code = filename.split('_')[0]

    print(f"\nProcessing {filename} ({currency_code})...")

    # Skip USD since it's the base currency
    if currency_code == 'USD':
        print(f"Skipping USD (base currency)")
        continue

    # Get the last date from the existing CSV
    last_date = get_last_date_from_csv(csv_file)
    if last_date is None:
        print(f"Could not read last date from {filename}")
        failed_files.append(filename)
        continue

    last_date = last_date.date()
    print(f"Last date in file: {last_date}")

    # Check if we need to fetch new data
    if last_date >= target_date:
        print(f"File is already up to date (last date: {last_date})")
        continue

    # Fetch missing rates
    missing_rates = fetch_missing_rates(currency_code, pd.to_datetime(last_date), pd.to_datetime(target_date))

    if missing_rates is None or missing_rates.empty:
        print(f"No new data to add for {currency_code}")
        continue

    try:
        # Read existing data
        existing_df = pd.read_csv(csv_file)

        # Append new data
        updated_df = pd.concat([existing_df, missing_rates], ignore_index=True)

        # Save back to file
        updated_df.to_csv(csv_file, index=False)

        print(f"✓ Added {len(missing_rates)} new records to {filename}")
        updated_files.append((filename, len(missing_rates)))

    except Exception as e:
        print(f"✗ Error updating {filename}: {e}")
        failed_files.append(filename)

print(f"\n=== UPDATE SUMMARY ===")
print(f"Successfully updated {len(updated_files)} files:")
for filename, count in updated_files:
    print(f"  {filename}: +{count} records")

if failed_files:
    print(f"\nFailed to update {len(failed_files)} files:")
    for filename in failed_files:
        print(f"  {filename}")
else:
    print(f"\nAll files processed successfully!")


Processing JOD_USD_rates.csv (JOD)...
Last date in file: 2025-10-24
Fetching JOD rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to JOD_USD_rates.csv

Processing JPY_USD_rates.csv (JPY)...
Last date in file: 2025-10-24
Fetching JPY rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to JPY_USD_rates.csv

Processing NOK_USD_rates.csv (NOK)...
Last date in file: 2025-10-24
Fetching NOK rates from 2025-10-25 to 2025-10-28


/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future 

✓ Added 2 new records to NOK_USD_rates.csv

Processing PEN_USD_rates.csv (PEN)...
Last date in file: 2025-10-24
Fetching PEN rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to PEN_USD_rates.csv

Processing SVC_USD_rates.csv (SVC)...
Last date in file: 2025-10-24
Fetching SVC rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to SVC_USD_rates.csv

Processing MAD_USD_rates.csv (MAD)...
Last date in file: 2025-10-24
Fetching MAD rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to MAD_USD_rates.csv

Processing ILS_USD_rates.csv (ILS)...
Last date in file: 2025-10-24
Fetching ILS rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to ILS_USD_rates.csv

Processing IQD_USD_rates.csv (IQD)...
Last date in file: 2025-10-24
Fetching IQD rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to IQD_USD_rates.csv

Processing BHD_USD_rates.csv (BHD)...
Last date in file: 2025-10-24
Fetching BHD rates from 2025-10-25 to 2025-10-28


/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future 

✓ Added 2 new records to BHD_USD_rates.csv

Processing GNF_USD_rates.csv (GNF)...
Last date in file: 2025-10-24
Fetching GNF rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to GNF_USD_rates.csv

Processing THB_USD_rates.csv (THB)...
Last date in file: 2025-10-24
Fetching THB rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to THB_USD_rates.csv

Processing GBP_USD_rates.csv (GBP)...
Last date in file: 2025-10-24
Fetching GBP rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to GBP_USD_rates.csv

Processing BMD_USD_rates.csv (BMD)...
Last date in file: 2025-10-24
Fetching BMD rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to BMD_USD_rates.csv

Processing KZT_USD_rates.csv (KZT)...
Last date in file: 2025-10-24
Fetching KZT rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to KZT_USD_rates.csv

Processing NZD_USD_rates.csv (NZD)...
Last date in file: 2025-10-24
Fetching NZD rates from 2025-10-25 to 2025-10-28


...

/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future 

✓ Added 2 new records to CNY_USD_rates.csv

Processing IDR_USD_rates.csv (IDR)...
Last date in file: 2025-10-24
Fetching IDR rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to IDR_USD_rates.csv

Processing BSD_USD_rates.csv (BSD)...
Last date in file: 2025-10-24
Fetching BSD rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to BSD_USD_rates.csv

Processing RUB_USD_rates.csv (RUB)...
Last date in file: 2025-10-24
Fetching RUB rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to RUB_USD_rates.csv

Processing EGP_USD_rates.csv (EGP)...
Last date in file: 2025-10-24
Fetching EGP rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to EGP_USD_rates.csv

Processing UZS_USD_rates.csv (UZS)...
Last date in file: 2025-10-24
Fetching UZS rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to UZS_USD_rates.csv

Processing AED_USD_rates.csv (AED)...
Last date in file: 2025-10-24
Fetching AED rates from 2025-10-25 to 2025-10-28


/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future 

✓ Added 2 new records to AED_USD_rates.csv

Processing PHP_USD_rates.csv (PHP)...
Last date in file: 2025-10-25
Fetching PHP rates from 2025-10-26 to 2025-10-28
✓ Added 2 new records to PHP_USD_rates.csv

Processing ZAR_USD_rates.csv (ZAR)...
Last date in file: 2025-10-24
Fetching ZAR rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to ZAR_USD_rates.csv

Processing CLP_USD_rates.csv (CLP)...
Last date in file: 2025-10-24
Fetching CLP rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to CLP_USD_rates.csv

Processing MXN_USD_rates.csv (MXN)...
Last date in file: 2025-10-24
Fetching MXN rates from 2025-10-25 to 2025-10-28
✓ Added 2 new records to MXN_USD_rates.csv

Processing AMD_USD_rates.csv (AMD)...
Last date in file: 2025-10-24
Fetching AMD rates from 2025-10-25 to 2025-10-28
✓ Added 1 new records to AMD_USD_rates.csv

=== UPDATE SUMMARY ===
Successfully updated 56 files:
  JOD_USD_rates.csv: +2 records
  JPY_USD_rates.csv: +2 records
  NOK_USD_rates.csv: +2 records
 

/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/nm/cck9mh6514n_0nn8t1qbh5hm0000gn/T/ipykernel_83303/1566636401.py:17: FutureWarning: In a future 